# Problem 3

Use this notebook to write your code for problem 3.

In [11]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## 3D - Convolutional network

As in problem 2, we have conveniently provided for your use code that loads and preprocesses the MNIST data.

In [12]:
# load MNIST data into PyTorch format
import torch
import torchvision
import torchvision.transforms as transforms

# set batch size
batch_size = 32

# load training data downloaded into data/ folder
mnist_training_data = torchvision.datasets.MNIST('data/', train=True, download=True,
                                                transform=transforms.ToTensor())
# transforms.ToTensor() converts batch of images to 4-D tensor and normalizes 0-255 to 0-1.0
training_data_loader = torch.utils.data.DataLoader(mnist_training_data,
                                                  batch_size=batch_size,
                                                  shuffle=True)

# load test data
mnist_test_data = torchvision.datasets.MNIST('data/', train=False, download=True,
                                                transform=transforms.ToTensor())
test_data_loader = torch.utils.data.DataLoader(mnist_test_data,
                                                  batch_size=batch_size,
                                                  shuffle=False)

In [13]:
# look at the number of batches per epoch for training and validation
print(f'{len(training_data_loader)} training batches')
print(f'{len(training_data_loader) * batch_size} training samples')
print(f'{len(test_data_loader)} validation batches')

1875 training batches
60000 training samples
313 validation batches


In [14]:
# sample model
import torch.nn as nn

model = nn.Sequential(
    nn.Conv2d(1, 8, kernel_size=(3,3)),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Dropout(p=0.5),
    
    nn.Conv2d(8, 8, kernel_size=(3,3)),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Dropout(p=0.5),
    
    nn.Flatten(),
    nn.Linear(25*8, 64),
    nn.ReLU(),
    nn.Linear(64, 10)
    # PyTorch implementation of cross-entropy loss includes softmax layer
)

In [15]:
# why don't we take a look at the shape of the weights for each layer
for p in model.parameters():
    print(p.data.shape)

torch.Size([8, 1, 3, 3])
torch.Size([8])
torch.Size([8, 8, 3, 3])
torch.Size([8])
torch.Size([64, 200])
torch.Size([64])
torch.Size([10, 64])
torch.Size([10])


In [16]:
# our model has some # of parameters:
count = 0
for p in model.parameters():
    n_params = np.prod(list(p.data.shape)).item()
    count += n_params
print(f'total params: {count}')

total params: 14178


In [17]:
# For a multi-class classification problem
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters())

In [18]:
# Train the model for 10 epochs, iterating on the data in batches
n_epochs = 10

# store metrics
training_accuracy_history = np.zeros([n_epochs, 1])
training_loss_history = np.zeros([n_epochs, 1])
validation_accuracy_history = np.zeros([n_epochs, 1])
validation_loss_history = np.zeros([n_epochs, 1])

for epoch in range(n_epochs):
    print(f'Epoch {epoch+1}/10:', end='')
    train_total = 0
    train_correct = 0
    # train
    model.train()
    for i, data in enumerate(training_data_loader):
        images, labels = data
        optimizer.zero_grad()
        # forward pass
        output = model(images)
        # calculate categorical cross entropy loss
        loss = criterion(output, labels)
        # backward pass
        loss.backward()
        optimizer.step()
        
        # track training accuracy
        _, predicted = torch.max(output.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()
        # track training loss
        training_loss_history[epoch] += loss.item()
        # progress update after 180 batches (~1/10 epoch for batch size 32)
        if i % 180 == 0: print('.',end='')
    training_loss_history[epoch] /= len(training_data_loader)
    training_accuracy_history[epoch] = train_correct / train_total
    print(f'\n\tloss: {training_loss_history[epoch,0]:0.4f}, acc: {training_accuracy_history[epoch,0]:0.4f}',end='')
        
    # validate
    test_total = 0
    test_correct = 0
    with torch.no_grad():
        model.eval()
        for i, data in enumerate(test_data_loader):
            images, labels = data
            # forward pass
            output = model(images)
            # find accuracy
            _, predicted = torch.max(output.data, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()
            # find loss
            loss = criterion(output, labels)
            validation_loss_history[epoch] += loss.item()
        validation_loss_history[epoch] /= len(test_data_loader)
        validation_accuracy_history[epoch] = test_correct / test_total
    print(f', val loss: {validation_loss_history[epoch,0]:0.4f}, val acc: {validation_accuracy_history[epoch,0]:0.4f}')

Epoch 1/10:...........
	loss: 0.6748, acc: 0.7780, val loss: 0.2176, val acc: 0.9495
Epoch 2/10:...........
	loss: 0.4449, acc: 0.8588, val loss: 0.1928, val acc: 0.9538
Epoch 3/10:...........
	loss: 0.4185, acc: 0.8693, val loss: 0.1847, val acc: 0.9502
Epoch 4/10:...........
	loss: 0.4085, acc: 0.8736, val loss: 0.1654, val acc: 0.9597
Epoch 5/10:...........
	loss: 0.4074, acc: 0.8726, val loss: 0.1979, val acc: 0.9474
Epoch 6/10:...........
	loss: 0.4021, acc: 0.8771, val loss: 0.1585, val acc: 0.9588
Epoch 7/10:...........
	loss: 0.3966, acc: 0.8782, val loss: 0.1748, val acc: 0.9597
Epoch 8/10:...........
	loss: 0.3890, acc: 0.8817, val loss: 0.1965, val acc: 0.9526
Epoch 9/10:...........
	loss: 0.3931, acc: 0.8802, val loss: 0.1793, val acc: 0.9540
Epoch 10/10:...........
	loss: 0.3948, acc: 0.8796, val loss: 0.1499, val acc: 0.9636


Above, we output the training loss/accuracy as well as the validation loss and accuracy. Not bad! Let's see if you can do better.

In [19]:

# Build a deeper CNN with >= 1 dropout + 1 batchnorm layer
model = nn.Sequential(
    # Conv block 1
    nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1),
    nn.BatchNorm2d(16),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),     
    nn.Dropout(p=0.3),

    # Conv block 2
    nn.Conv2d(16, 32, kernel_size=3, padding=1),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),      
    nn.Dropout(p=0.3),

    # Optionally add a 3rd block if you want even better accuracy:
    nn.Conv2d(32, 64, kernel_size=3, padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, padding=1), 
    nn.Dropout(p=0.3),
    
    nn.Flatten(),                    

    nn.Linear(in_features=64*4*4, out_features=256),
    nn.BatchNorm1d(256),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(256, 10)              
)

# Loss function (includes softmax internally) and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(model.parameters(), lr=1e-3)

# Training loop 
n_epochs = 10
for epoch in range(n_epochs):
    model.train()  
    for batch_idx, (images, labels) in enumerate(training_data_loader):
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
    
    # Validation pass
    model.eval()
    correct = 0
    total = 0
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in test_data_loader:
            logits = model(images)
            val_loss += criterion(logits, labels).item()
            preds = logits.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    val_accuracy = correct / total
    val_loss /= len(test_data_loader)
    print(f"Epoch {epoch+1}/{n_epochs} | Val Loss = {val_loss:.4f}, Val Acc = {val_accuracy:.4f}")


Epoch 1/10 | Val Loss = 0.0450, Val Acc = 0.9857
Epoch 2/10 | Val Loss = 0.0329, Val Acc = 0.9889
Epoch 3/10 | Val Loss = 0.0293, Val Acc = 0.9898
Epoch 4/10 | Val Loss = 0.0264, Val Acc = 0.9916
Epoch 5/10 | Val Loss = 0.0241, Val Acc = 0.9919
Epoch 6/10 | Val Loss = 0.0232, Val Acc = 0.9930
Epoch 7/10 | Val Loss = 0.0230, Val Acc = 0.9929
Epoch 8/10 | Val Loss = 0.0235, Val Acc = 0.9932
Epoch 9/10 | Val Loss = 0.0226, Val Acc = 0.9939
Epoch 10/10 | Val Loss = 0.0219, Val Acc = 0.9932
